<a href="https://colab.research.google.com/github/Swastik200/30DaysOfDSA/blob/main/Input_Attn_Upgrad4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


In [2]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from math import sqrt
from keras.models import Sequential
from keras.layers import LSTM, Dense, Attention
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!cp "/content/drive/MyDrive/CMAPSS.zip" "/content"
!unzip CMAPSS.zip

Archive:  CMAPSS.zip
  inflating: CMAPSS/readme.txt       
  inflating: CMAPSS/RUL_FD001.txt    
  inflating: CMAPSS/RUL_FD002.txt    
  inflating: CMAPSS/RUL_FD003.txt    
  inflating: CMAPSS/RUL_FD004.txt    
  inflating: CMAPSS/test_FD001.txt   
  inflating: CMAPSS/test_FD002.txt   
  inflating: CMAPSS/test_FD003.txt   
  inflating: CMAPSS/test_FD004.txt   
  inflating: CMAPSS/train_FD001.txt  
  inflating: CMAPSS/train_FD002.txt  
  inflating: CMAPSS/train_FD003.txt  
  inflating: CMAPSS/train_FD004.txt  
  inflating: CMAPSS/x.txt            


In [5]:
index_names = ['unit_number', 'time_cycles']
setting_names = ['setting_1', 'setting_2', 'setting_3']
sensor_names = ['sensor_{}'.format(i) for i in range(1,22)]
col_names = index_names + setting_names + sensor_names

In [6]:
path = '/content/CMAPSS/'

In [61]:
train_df = pd.read_csv(path+'train_FD004.txt', delimiter=' ', header=None)
columns = ['unit_number', 'time_in_cycles', 'operational_setting_1', 'operational_setting_2', 'operational_setting_3'] + [f'sensor_{i}' for i in range(1, 24)]
train_df.columns = columns

In [62]:
max_cycles = train_df.groupby('unit_number')['time_in_cycles'].max().reset_index()
max_cycles.columns = ['unit_number', 'max_cycles']
train_df = train_df.merge(max_cycles, on='unit_number', how='left')
train_df['RUL'] = train_df['max_cycles'] - train_df['time_in_cycles']
train_df.drop('max_cycles', axis=1, inplace=True)

In [63]:
def prepare_sequences(data, n_steps):
    X, y = [], []
    for i in range(len(data)):
        end_ix = i + n_steps
        if end_ix > len(data)-1:
            break
        seq_x, seq_y = data[i:end_ix, :-1], data[end_ix, -1]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [64]:
n_steps = 20


In [65]:
data = train_df.drop(columns=['unit_number', 'time_in_cycles']).values

In [66]:
X, y = prepare_sequences(data, n_steps)

In [67]:
from keras.models import Model
from keras.layers import LSTM, Dense, Attention, Input

# Define input tensors
query_input = Input(shape=(None, 100))  # Shape depends on the output shape of the LSTM layer
value_input = Input(shape=(None, 100))  # Shape depends on the output shape of the LSTM layer

# Define and train Input-Attn-LSTM model
attention_output = Attention()([query_input, value_input])
output = Dense(1)(attention_output)

# Define the model
model = Model(inputs=[query_input, value_input], outputs=output)




In [68]:
from keras.models import Model
from keras.layers import LSTM, Dense, Attention, Input, TimeDistributed

# Define input tensor
input_sequence = Input(shape=(n_steps, X.shape[2]))  # Shape: (None, 20, 26)

# Define and train Input-Attn-LSTM model
lstm_output = LSTM(100, activation='relu', return_sequences=True)(input_sequence)
repeated_lstm_output = TimeDistributed(Dense(100))(lstm_output)  # Shape: (None, 20, 100)
attention_output = Attention()([repeated_lstm_output, repeated_lstm_output])
output = Dense(1)(attention_output)

# Define the model
model = Model(inputs=input_sequence, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X, y, epochs=100, verbose=0)


In [69]:
train_pred = model.predict(X)

1914/1914 [==============================] - 15s 8ms/step


In [70]:
from math import sqrt
from sklearn.metrics import mean_squared_error
import numpy as np

In [71]:
if np.isnan(train_pred).any():
    # Replace NaN values with zeros, you can use other strategies based on your data
    train_pred[np.isnan(train_pred)] = 0

In [72]:
train_pred_aggregated = train_pred.mean(axis=1)  # Take the mean along the sequence axis

# Ensure that train_pred_aggregated has the same shape as y
train_pred_aggregated = train_pred_aggregated.reshape(-1)

In [73]:
train_rmse = sqrt(mean_squared_error(y, train_pred_aggregated))
print(f"Train RMSE: {train_rmse}")

Train RMSE: 160.65427149660647


In [74]:
from sklearn.metrics import mean_absolute_error, r2_score

In [75]:
from sklearn.metrics import mean_absolute_error, r2_score

# Aggregate predictions to get a single prediction per sample
train_pred_aggregated = train_pred.mean(axis=1)  # Take the mean along the sequence axis

# Ensure that train_pred_aggregated has the same shape as y
train_pred_aggregated = train_pred_aggregated.reshape(-1)

# Calculate mean absolute error (MAE)
train_mae = mean_absolute_error(y, train_pred_aggregated)

# Calculate R2 score
train_r2 = r2_score(y, train_pred_aggregated)

print(f"Train R2: {train_r2}, MAE: {train_mae}")


Train R2: -2.2048762750881226, MAE: 133.2535399892208


In [78]:
test_df = pd.read_csv(path+'test_FD004.txt', delimiter=' ', header=None)
test_df.columns = columns

In [79]:
max_cycles_test = test_df.groupby('unit_number')['time_in_cycles'].max().reset_index()
max_cycles_test.columns = ['unit_number', 'max_cycles']
test_df = test_df.merge(max_cycles_test, on='unit_number', how='left')
test_df['RUL'] = test_df['max_cycles'] - test_df['time_in_cycles']
test_df.drop('max_cycles', axis=1, inplace=True)


In [80]:
data_test = test_df.drop(columns=['unit_number', 'time_in_cycles']).values
X_test, y_test = prepare_sequences(data_test, n_steps)


In [81]:
test_pred = model.predict(X_test)

1288/1288 [==============================] - 10s 8ms/step


In [82]:
if np.isnan(test_pred).any():
    # Replace NaN values with zeros, you can use other strategies based on your data
    test_pred[np.isnan(test_pred)] = 0

In [83]:
test_pred_aggregated = test_pred.mean(axis=1)  # Take the mean along the sequence axis

# Ensure that train_pred_aggregated has the same shape as y
test_pred_aggregated = test_pred_aggregated.reshape(-1)

In [84]:
from math import sqrt
from sklearn.metrics import mean_squared_error

# Ensure that the dimensions of y_test and test_pred match
test_pred_reshaped = test_pred.reshape(-1, 1)[:len(y_test)]

# Calculate RMSE for the test set
test_rmse = sqrt(mean_squared_error(y_test, test_pred_reshaped))
print(f"Test RMSE: {test_rmse}")


Test RMSE: 136.3883626944313


In [86]:
from sklearn.metrics import mean_absolute_error, r2_score

# Ensure that the dimensions of y_test and test_pred match
test_pred_reshaped = test_pred.reshape(-1, 1)[:len(y_test)]

# Calculate mean absolute error (MAE)
test_mae = mean_absolute_error(y_test, test_pred_reshaped)

# Calculate R2 score
test_r2 = r2_score(y_test, test_pred_reshaped)

print(f"Test R2: {test_r2}, MAE: {test_mae}")


Test R2: -1.6552122019752518, MAE: 107.68483274263242
